In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#Loading data
url = "http://www0.cs.ucl.ac.uk/staff/M.Herbster/boston-filter/Boston-filtered.csv"
data = pd.read_csv(url)
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [3]:
data = data.to_numpy()

In [4]:
#Splitting data into X and Y
x = data[:,0:12]
y = data[:,12].reshape(506,1)

#Randomly splitting X and Y into test and train data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.33, random_state =42)

In [5]:
#initialising K 
K = np.zeros((339,339))

In [6]:
nrows, ncols = x_train.shape
print(nrows)

339


In [7]:
#Creating sigma vector 
sigma = np.zeros((13,1))
sigmashape = sigma.shape
sigmapowers = np.arange(7,13.5,0.5)

for i in range(sigmashape[0]):
    sigma[i][0] = 2**sigmapowers[i]
    
#Creating gamma vector 
gamma = np.zeros((15,1))
gammashape = gamma.shape
gammapowers = np.arange(-40, -25, 1, dtype=float)

for i in range(gammashape[0]):
    gamma[i][0] = 2**gammapowers[i]

In [8]:
#filling kernel function 
#I should have 13 Kernel matrices because of 13sigmas
#K_all is an array that contains the 13 Kernel matrices
K_all = []

for sigma in sigma:
    K.fill(0)
    for i in range(nrows):
        for j in range(nrows):
            K[i][j] = np.exp((-np.linalg.norm((x_train[i] - x_train[j]))**2)/(2*sigma**2))
    K_all.append(K)

In [9]:
#Should have 15 a's per Kernel matrix as there are 15 gammas
#Will result in 195 a's
a_all = np.zeros((13,15), dtype = object)
I = np.identity(339)

for i in range(len(K_all)):
    for j in range(gammashape[0]):
        a_all[i][j] = np.linalg.inv((K_all[i] + (gamma[j][0]*nrows*I)))@y_train


In [10]:
#Now that we have all of our combinations of alpha, we can run predictions
#Before we can run predictions, we have to run kernel function on 
yhat = np.zeros((167,1))
yhat_all_a = np.zeros((13,15), dtype = object)

for i in range(yhat_all_a.shape[0]):
    for j in range(yhat_all_a.shape[1]):
        yhat_all_a[i][j] = np.zeros((167,1))


In [11]:
for a in range(a_all.shape[0]):
    for b in range(a_all.shape[1]):
        for i in range(x_test.shape[0]):
            for j in range(nrows):
                yhat_all_a[a][b][i] += a_all[a][b][j]*np.exp((-np.linalg.norm((x_train[j] - x_test[i]))**2)/(2*sigma**2))

In [12]:
#Mean squared error function
def MSEnp(y,yhat,n):
    sqrdiff = (y - yhat)**2
    SSE = np.sum(sqrdiff)
    meanSE = SSE/n
    return meanSE

In [13]:
yhat_all_a[0][0]

array([[26.4474678 ],
       [32.71352005],
       [16.78098106],
       [24.95278358],
       [16.74769211],
       [21.86612892],
       [16.84768677],
       [15.98775673],
       [22.40632629],
       [19.16090393],
       [22.75573349],
       [16.91973495],
       [-0.20640755],
       [20.70692825],
       [16.77319717],
       [22.91338158],
       [19.17901421],
       [ 6.54673386],
       [43.30149078],
       [15.49092484],
       [26.25774956],
       [27.67129517],
       [12.42403412],
       [22.25772667],
       [19.19817734],
       [17.35971069],
       [21.79246902],
       [14.05774879],
       [20.31596947],
       [17.76173592],
       [18.76909637],
       [24.50315857],
       [20.89295197],
       [24.35227394],
       [14.94663811],
       [17.91466904],
       [32.23353577],
       [20.01119804],
       [22.48842812],
       [25.94922066],
       [13.24411392],
       [31.1440258 ],
       [46.07964516],
       [16.1537323 ],
       [25.39368439],
       [18

In [14]:
MSE_all_a = np.zeros((13,15))

for i in range(yhat_all_a.shape[0]):
    for j in range(yhat_all_a.shape[1]):
        MSE_all_a[i][j] = MSEnp(y_test, yhat_all_a[i][j],167)

In [15]:
np.argmin(MSE_all_a)

0

In [16]:
#Smallest MSE is MSE[0,0], this index corresponds to [0,0] of sigmas and gammas, the 0th sigma and 0th gamma
#This is sigma:2**7, gamma: 2**-40

In [17]:
MSE_all_a

array([[13.20801214, 14.39378913, 16.0362285 , 17.71579139, 19.08050614,
        20.06703294, 20.75848984, 21.20840373, 21.44607551, 21.54041829,
        21.60243159, 21.76699811, 22.16630686, 22.87004874, 23.8805791 ],
       [13.20801214, 14.39378913, 16.0362285 , 17.71579139, 19.08050614,
        20.06703294, 20.75848984, 21.20840373, 21.44607551, 21.54041829,
        21.60243159, 21.76699811, 22.16630686, 22.87004874, 23.8805791 ],
       [13.20801214, 14.39378913, 16.0362285 , 17.71579139, 19.08050614,
        20.06703294, 20.75848984, 21.20840373, 21.44607551, 21.54041829,
        21.60243159, 21.76699811, 22.16630686, 22.87004874, 23.8805791 ],
       [13.20801214, 14.39378913, 16.0362285 , 17.71579139, 19.08050614,
        20.06703294, 20.75848984, 21.20840373, 21.44607551, 21.54041829,
        21.60243159, 21.76699811, 22.16630686, 22.87004874, 23.8805791 ],
       [13.20801214, 14.39378913, 16.0362285 , 17.71579139, 19.08050614,
        20.06703294, 20.75848984, 21.20840373, 

In [18]:
#Ideal sigma and gamma was calculated for test set, with MSE of 13.2 
#Has to be also done for training set
yhat_all_a_train = np.zeros((13,15), dtype = object)

for i in range(yhat_all_a_train.shape[0]):
    for j in range(yhat_all_a_train.shape[1]):
        yhat_all_a_train[i][j] = np.zeros((339,1))
        
for a in range(a_all.shape[0]):
    for b in range(a_all.shape[1]):
        for i in range(x_train.shape[0]):
            for j in range(nrows):
                yhat_all_a_train[a][b][i] += a_all[a][b][j]*np.exp((-np.linalg.norm((x_train[j] - x_train[i]))**2)/(2*sigma**2))


In [21]:
MSE_all_a_train = np.zeros((13,15))

for i in range(yhat_all_a_train.shape[0]):
    for j in range(yhat_all_a_train.shape[1]):
        MSE_all_a_train[i][j] = MSEnp(y_train, yhat_all_a_train[i][j],339)

In [22]:
MSE_all_a_train

array([[11.88905787, 13.83147314, 16.10828008, 18.26906969, 19.98194038,
        21.23461684, 22.17556218, 22.91312658, 23.49196719, 23.96090394,
        24.40836496, 24.95899688, 25.73403037, 26.77446605, 28.04262246],
       [11.88905787, 13.83147314, 16.10828008, 18.26906969, 19.98194038,
        21.23461684, 22.17556218, 22.91312658, 23.49196719, 23.96090394,
        24.40836496, 24.95899688, 25.73403037, 26.77446605, 28.04262246],
       [11.88905787, 13.83147314, 16.10828008, 18.26906969, 19.98194038,
        21.23461684, 22.17556218, 22.91312658, 23.49196719, 23.96090394,
        24.40836496, 24.95899688, 25.73403037, 26.77446605, 28.04262246],
       [11.88905787, 13.83147314, 16.10828008, 18.26906969, 19.98194038,
        21.23461684, 22.17556218, 22.91312658, 23.49196719, 23.96090394,
        24.40836496, 24.95899688, 25.73403037, 26.77446605, 28.04262246],
       [11.88905787, 13.83147314, 16.10828008, 18.26906969, 19.98194038,
        21.23461684, 22.17556218, 22.91312658, 

In [23]:
#argmin of MSE array, returns [0,0], same as for test set. 
#the 0th sigma and 0th gamma produce the best results. 
# sigma: 2**7, gamma:2**-40
np.argmin(MSE_all_a_train)

0